In [119]:
import numpy as np
import math

In [50]:
def charToAscii(plaintext): # converts string to array of ascii values
  ascii = []
  for char in plaintext:
    ascii.append(ord(char))
  return np.array(ascii)

In [174]:
def asciiToChar(arr): # converts array of ascii values to string
  string = ""
  for a in arr:
    string = string + chr(a)
  return string

In [175]:
def addPadding(plaintext_ascii): # adds required padding -> padding done by adding 0s in the plain text
  l = len(plaintext_ascii)
  for i in range((4 - l%4)%4):
    plaintext_ascii = np.append(plaintext_ascii, 48)
  return plaintext_ascii

In [176]:
def inputKey():
  return np.array([[2, 3], [4, 5]])

In [177]:
def inputIV():
    return np.array([[4, 5], [7, 8]])

In [178]:
def checkInverse(matrix, modVal): # checks if inverse of matrix exists
  if math.gcd(matrix[0][0]*matrix[1][1] - matrix[0][1]*matrix[1][0], modVal) == 1:
    return True
  else:
    return False

In [179]:
def modulo(k, matrix): # modulo operation 255
  return matrix%k

In [180]:
def encryption(key, P, modVal): # hill cipher encryption
  return modulo(modVal, np.dot(key, P))

In [181]:
def preprocessPlainText(plaintext): # remove spaces
  return plaintext.replace(" ", "")

In [182]:
def plainTextToBlocks(processedPlainText, modVal): # converts preprocessed plain text to 2X2 blocks of ascii values
  asciiPaddedArray = addPadding(charToAscii(processedPlainText))
  i = 0
  plainTextBlocks = []
  while i < len(asciiPaddedArray):
    P = [[asciiPaddedArray[i], asciiPaddedArray[i+1]], [asciiPaddedArray[i+2], asciiPaddedArray[i+3]]]
    i += 4
    plainTextBlocks.append(P)
  return np.array(plainTextBlocks)

In [183]:
def blocksToPlainText(blocks): # converts 2X2 blocks of ascii values to text
  encryptedArray = []
  for block in blocks:
    encryptedArray.append(block[0][0])
    encryptedArray.append(block[0][1])
    encryptedArray.append(block[1][0])
    encryptedArray.append(block[1][1])
  textArray = np.array(encryptedArray)
  text = asciiToChar(textArray)
  return text

In [184]:
def cipherFeedbackModeEncrypt(plainTextBlocks, IV, key, modVal): # encryption in cipher feedback mode
  T = encryption(key, IV, modVal)
  C = np.bitwise_xor(T, plainTextBlocks[0])
  encryptedBlocks = []
  encryptedBlocks.append(C)
  for i in range(1, len(plainTextBlocks)):
    T = encryption(key, C, modVal)
    C = np.bitwise_xor(T, plainTextBlocks[i])
    encryptedBlocks.append(C)
  return np.array(encryptedBlocks)

In [185]:
def cipherFeedbackModeDecrypt(encryptedBlocks, IV, key, modVal): # decryption in cipher feedback mode
  i = len(encryptedBlocks) - 1
  decryptedTextBlocks = []
  while i > 0:
    T = encryption(key, encryptedBlocks[i-1], modVal)
    P = np.bitwise_xor(T, encryptedBlocks[i])
    decryptedTextBlocks.append(P)
    i -= 1
  T = encryption(key, IV, modVal)
  P = np.bitwise_xor(T, encryptedBlocks[0])
  decryptedTextBlocks.append(P)
  decryptedTextBlocks = np.flipud(decryptedTextBlocks)
  return np.array(decryptedTextBlocks)

In [186]:
def allOperations(plainText, key, IV, modVal, filename): # main function that performs all required operations
  txtfile = open(filename,"w")
  preprocessedText = preprocessPlainText(plainText)
  print(charToAscii(preprocessedText))
  txtfile.write(str(charToAscii(preprocessedText)))
  txtfile.write("\n")
  if checkInverse(key, modVal) == False:
    print("Key-inverse does not exist")
    txtfile.write("Key-inverse does not exist")
    txtfile.write("\n")
    return
  print("Plaintext: ", str(plainText))
  txtfile.write("Plaintext: " + str(plainText))
  txtfile.write("\n")
  plainTextBlocks= plainTextToBlocks(preprocessedText, modVal)
  encryptedBlocks = cipherFeedbackModeEncrypt(plainTextBlocks, IV, key, modVal)
  encryptedText = blocksToPlainText(encryptedBlocks)
  print("Encrypted text: ", encryptedText)
  txtfile.write("Encrypted text: " + encryptedText)
  txtfile.write("\n")
  decryptedBlocks = cipherFeedbackModeDecrypt(encryptedBlocks, IV, key, modVal)
  decryptedText = blocksToPlainText(decryptedBlocks)
  print("Decrypted text: ", decryptedText)
  txtfile.write("Decrypted text: " + decryptedText)
  txtfile.write("\n")

In [187]:
allOperations("Phishing", np.array([[2, 3], [4, 5]]), np.array([[4, 5], [7, 8]]), 256, "191IT207_IT352_P6_Sample_Output_TC1.txt")

[ 80 104 105 115 104 105 110 103]
Key-inverse does not exist


In [188]:
allOperations("SBINITKS", np.array([[3, 3], [4, 4]]), np.array([[3, 8], [3, 7]]), 256, "191IT207_IT352_P6_Sample_Output_TC2.txt")

[83 66 73 78 73 84 75 83]
Key-inverse does not exist


In [189]:
allOperations("decrease", np.array([[14, 13], [14, 15]]), np.array([[14, 5], [5, 8]]), 256, "191IT207_IT352_P6_Output_TC1.txt")

[100 101  99 114 101  97 115 101]
Key-inverse does not exist


In [190]:
allOperations("delicate", np.array([[13, 12], [14, 15]]), np.array([[13, 18], [18, 17]]), 256, "191IT207_IT352_P6_Output_TC2.txt")

[100 101 108 105  99  97 116 101]
Plaintext:  delicate
Encrypted text:  eS(bn*}
Decrypted text:  delicate


In [191]:
allOperations("adequate", np.array([[3, 2], [2, 5]]), np.array([[7, 8], [8, 7]]), 256, "191IT207_IT352_P6_Output_TC3.txt")

[ 97 100 101 113 117  97 116 101]
Plaintext:  adequate
Encrypted text:  DBSB+S+
Decrypted text:  adequate


In [192]:
allOperations("anything", np.array([[15, 13], [13, 15]]), np.array([[7, 8], [7, 7]]), 256, "191IT207_IT352_P6_Output_TC4.txt")

[ 97 110 121 116 104 105 110 103]
Key-inverse does not exist


In [193]:
allOperations("efficacy", np.array([[5, 17], [7, 15]]), np.array([[15, 6], [6, 15]]), 256, "191IT207_IT352_P6_Output_TC5.txt")

[101 102 102 105  99  97  99 121]
Key-inverse does not exist


In [194]:
allOperations("earnings", np.array([[5, 13], [3, 15]]), np.array([[17, 2], [6, 17]]), 256, "191IT207_IT352_P6_Output_TC6.txt")

[101  97 114 110 105 110 103 115]
Key-inverse does not exist
